# Tutorial 2: ZnO Wurtzite (0001) - Complete 4-Step phshift2007 Workflow

## Introduction

This tutorial provides a detailed walkthrough of the complete Barbieri/Van Hove phase shift calculation workflow using **Wurtzite Zinc Oxide (ZnO)** with the **(0001) Zn-terminated surface** as our example. This is a more complex system than the simple Ni(111) surface, involving:

- A **compound semiconductor** with two elements (Zn and O)
- **Non-cubic crystal structure** (hexagonal wurtzite)
- **Polar surface** (Zn-terminated vs O-terminated)

We will go through each of the four steps in the phshift2007 package individually, explaining the physics and intermediate outputs at each stage.

### Why ZnO?

ZnO is a widely studied transparent conducting oxide with applications in:
- Photovoltaics and solar cells
- Gas sensors
- Catalysis
- Piezoelectric devices

Understanding its surface structure via LEED is crucial for these applications.

### Wurtzite Crystal Structure

ZnO crystallizes in the wurtzite structure:
- Space group: P6₃mc (No. 186)
- Lattice parameters: a = 3.25 Angstrom, c = 5.21 Angstrom
- Each Zn atom is tetrahedrally coordinated to 4 O atoms
- The (0001) surface exposes Zn atoms (Zn-terminated)
- The (000-1) surface exposes O atoms (O-terminated)

### References

- Dulub, O., Boatner, L.A. & Diebold, U. (2002). "STM study of the geometric and electronic structure of ZnO(0001)-Zn, (000-1)-O, (10-10), and (11-20) surfaces." [Surf. Sci. 519, 201-217](https://doi.org/10.1016/S0039-6028(02)02211-2)
- Barbieri, A. & Van Hove, M.A. Phase shift calculation package: http://www.icts.hkbu.edu.hk/VanHove_files/leed/leedpack.html

## Setup

In [ ]:
# Install required packages
%pip install -q phaseshifts numpy matplotlib ase

In [ ]:
import os
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# phaseshifts imports
import phaseshifts
from phaseshifts import atorb, model
from phaseshifts.conphas import Conphas
from phaseshifts.leed import Converter, CLEEDInputValidator

# Set up plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print(f"phaseshifts version: {phaseshifts.__version__}")

In [ ]:
# Create working directory
work_dir = tempfile.mkdtemp(prefix='phaseshifts_zno_')
print(f"Working directory: {work_dir}")

## Visualizing the ZnO Wurtzite Structure

Let's first visualize the crystal structure to understand what we're calculating.

In [ ]:
try:
    from ase import Atoms
    from ase.build import bulk, surface
    from ase.visualize.plot import plot_atoms
    
    # ZnO Wurtzite structure parameters
    a_zno = 3.25  # Angstrom
    c_zno = 5.21  # Angstrom
    u_param = 0.382  # Internal parameter (ideal wurtzite: 0.375)
    
    # Create wurtzite ZnO bulk structure
    # Wurtzite has 4 atoms per unit cell: 2 Zn + 2 O
    zno_bulk = Atoms(
        symbols=['Zn', 'Zn', 'O', 'O'],
        positions=[
            [0, 0, 0],
            [a_zno/2, a_zno/(2*np.sqrt(3)), c_zno/2],
            [0, 0, u_param * c_zno],
            [a_zno/2, a_zno/(2*np.sqrt(3)), (0.5 + u_param) * c_zno]
        ],
        cell=[
            [a_zno, 0, 0],
            [a_zno/2, a_zno*np.sqrt(3)/2, 0],
            [0, 0, c_zno]
        ],
        pbc=True
    )
    
    # Create a (0001) slab (Zn-terminated)
    # For visualization, we'll create a 3x3 supercell with 4 layers
    zno_bulk_3x3 = zno_bulk.repeat((3, 3, 4))
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Color scheme: Zn = gray, O = red
    colors_bulk = ['gray' if s == 'Zn' else 'red' for s in zno_bulk_3x3.get_chemical_symbols()]
    
    # Top view (0001)
    plot_atoms(zno_bulk_3x3, axes[0], rotation='0x,0y,0z', colors=colors_bulk)
    axes[0].set_title('ZnO Wurtzite - Top View (0001)')
    
    # Side view  
    plot_atoms(zno_bulk_3x3, axes[1], rotation='90x,0y,0z', colors=colors_bulk)
    axes[1].set_title('ZnO Wurtzite - Side View')
    
    # Perspective view
    plot_atoms(zno_bulk_3x3, axes[2], rotation='30x,30y,0z', colors=colors_bulk)
    axes[2].set_title('ZnO Wurtzite - Perspective')
    
    plt.tight_layout()
    plt.savefig(os.path.join(work_dir, 'zno_structure.png'), dpi=150)
    plt.show()
    
    # Print structure info
    print("\n=== ZnO Wurtzite Structure ===")
    print(f"Lattice parameters: a = {a_zno:.3f} A, c = {c_zno:.3f} A")
    print(f"c/a ratio: {c_zno/a_zno:.4f} (ideal wurtzite: 1.633)")
    print(f"Internal parameter u: {u_param:.3f} (ideal: 0.375)")
    print(f"Zn-O bond length: {u_param * c_zno:.3f} A")
    print("\nColor legend: Gray = Zn, Red = O")
    
except ImportError:
    print("ASE not available for structure visualization.")
    print("Install with: pip install ase")
    
    # Still define the parameters for later use
    a_zno = 3.25
    c_zno = 5.21
    u_param = 0.382

## Step 0: Atomic Orbital Charge Density Calculation (phsh0)

### Theory

The first step calculates the **atomic charge density** for each element using a **relativistic Dirac-Fock** self-consistent field method. This gives us the radial charge distribution $\rho(r)$ for isolated atoms.

For each element, we need to specify:
- Atomic number (Z)
- Nuclear charge
- Electronic configuration (shells, angular momenta, occupancies)

### Input: atorb files

The input specifies the electronic shell structure. For each shell, we provide:
- Principal quantum number (n)
- Orbital angular momentum (l)
- Total angular momentum (j = l +/- 1/2)
- Occupancy

In [ ]:
# Zinc electronic configuration: [Ar] 3d10 4s2
# Atomic number: 30

atorb_Zn = '''Zn
  30.0  30.0    21     0.1
   1  0  0  0.5   2.0
   2  0  0  0.5   2.0
   2  1  1  0.5   2.0
   2  1  1  1.5   4.0
   3  0  0  0.5   2.0
   3  1  1  0.5   2.0
   3  1  1  1.5   4.0
   3  2  2  1.5   4.0
   3  2  2  2.5   6.0
   4  0  0  0.5   2.0
'''

# Oxygen electronic configuration: [He] 2s2 2p4
# Atomic number: 8

atorb_O = '''O
   8.0   8.0     6     0.1
   1  0  0  0.5   2.0
   2  0  0  0.5   2.0
   2  1  1  0.5   1.333333
   2  1  1  1.5   2.666667
'''

print("=== Zinc atorb input ===")
print("Format: n  l  j_num  j  occupancy")
print(atorb_Zn)

print("\n=== Oxygen atorb input ===")
print(atorb_O)

print("\nNote: For partially filled shells, occupancies are distributed")
print("proportionally between j = l+1/2 and j = l-1/2 sub-levels.")

In [ ]:
# Write atorb input files
atorb_Zn_file = os.path.join(work_dir, 'atorb_Zn.txt')
atorb_O_file = os.path.join(work_dir, 'atorb_O.txt')

with open(atorb_Zn_file, 'w') as f:
    f.write(atorb_Zn)
    
with open(atorb_O_file, 'w') as f:
    f.write(atorb_O)

print(f"Created: {atorb_Zn_file}")
print(f"Created: {atorb_O_file}")

In [ ]:
# Run Step 0: Calculate atomic charge densities
print("=" * 60)
print("STEP 0: Atomic Orbital Charge Density Calculation")
print("=" * 60)

try:
    print("\nCalculating charge density for Zn...")
    at_Zn_file = atorb.Atorb.calculate_Q_density(
        element='Zn',
        output_dir=work_dir
    )
    print(f"Output: {at_Zn_file}")
    
    print("\nCalculating charge density for O...")
    at_O_file = atorb.Atorb.calculate_Q_density(
        element='O',
        output_dir=work_dir
    )
    print(f"Output: {at_O_file}")
    
except Exception as e:
    print(f"\nNote: Calculation requires compiled Fortran library.")
    print(f"Error: {e}")
    print("\nFor demonstration, we'll create placeholder files.")
    
    at_Zn_file = os.path.join(work_dir, 'at_Zn.i')
    at_O_file = os.path.join(work_dir, 'at_O.i')

### Output: Atomic charge density files (at_*.i)

The output file contains the radial charge density on a mesh of points:

```
   r(1)  rho(r(1))
   r(2)  rho(r(2))
   ...
```

The charge density $\rho(r)$ is used in Step 1 to construct the muffin-tin potential.

## Step 1: Muffin-Tin Potential Calculation (phsh1)

### Theory

The **muffin-tin approximation** divides space into:
1. **Atomic spheres** ("muffin tins") where the potential is spherically symmetric
2. **Interstitial region** where the potential is constant (the "muffin-tin zero")

The potential inside each sphere is constructed by:
1. Superposing atomic charge densities from neighboring atoms
2. Solving the Poisson equation for the electrostatic potential
3. Adding an exchange-correlation term (Slater X-alpha approximation)

### Input Files

We need two types of input:

1. **Cluster file (cluster.i)**: Defines the atomic positions and cell
2. **Atomic file (atomic.i)**: Contains concatenated atomic charge densities

In [ ]:
# Define the ZnO(0001) CLEED input files

# Bulk file
zno_bulk_content = f'''# ZnO Wurtzite (0001) Zn-terminated - Bulk geometry
c: ZnO(0001)-Zn - Wurtzite Zinc Oxide, Zn-terminated
#
# Lattice parameters: a = {a_zno} A, c = {c_zno} A
# Hexagonal surface unit cell
#
a1:   {a_zno/2:.6f}   {-a_zno*np.sqrt(3)/2:.6f}   0.0000
a2:   {a_zno/2:.6f}    {a_zno*np.sqrt(3)/2:.6f}   0.0000
a3:   0.0000    0.0000   {-c_zno:.6f}
#
# Superstructure matrix (1x1)
m1:  1. 0.
m2:  0. 1.
#
# Search symmetry (3-fold rotational for hexagonal)
sr: 3  0.0  0.0
#
# Optical potential (eV)
# vr: inner potential (typical: -8 to -12 eV for oxides)
# vi: imaginary part (absorption, typical: 3-5 eV)
vr:   -10.00
vi:     4.00
#
# Bulk atomic positions
# Wurtzite basis: Zn at (0,0,0) and (1/3, 2/3, 1/2)
#                 O at (0,0,u) and (1/3, 2/3, 1/2+u)
#
# Layer 1: Zn at origin
pb: Zn_BVH  0.0000   0.0000   0.0000  dr3 0.025 0.025 0.025
pb: O_BVH   0.0000   0.0000   {u_param * c_zno:.4f}  dr3 0.025 0.025 0.025
# Layer 2: Zn-O pair at (1/3, 2/3, 1/2)
pb: Zn_BVH  {a_zno/2:.4f}   {a_zno/(2*np.sqrt(3)):.4f}   {c_zno/2:.4f}  dr3 0.025 0.025 0.025
pb: O_BVH   {a_zno/2:.4f}   {a_zno/(2*np.sqrt(3)):.4f}   {(0.5 + u_param) * c_zno:.4f}  dr3 0.025 0.025 0.025
#
# Energy range (eV)
ei: 30.
ef: 400.
es: 5.
#
# Other parameters
it: 0.
ip: 0.
ep: 1.e-2
#
# Maximum angular momentum
lm: 10
'''

# Slab file (Zn-terminated surface)
zno_slab_content = f'''# ZnO(0001) Zn-terminated - Slab geometry
#
# 2D surface lattice vectors
a1:   {a_zno/2:.6f}   {a_zno*np.sqrt(3)/2:.6f}   0.0000
a2:   {a_zno/2:.6f}   {-a_zno*np.sqrt(3)/2:.6f}   0.0000
#
m1:  1. 0.
m2:  0. 1.
#
# Surface/overlayer atomic positions (Zn-terminated)
# Top layer: Zn atoms
po: Zn_BVH  0.0000   0.0000   {2*c_zno:.4f}  dr3 0.025 0.025 0.025
po: O_BVH   0.0000   0.0000   {2*c_zno - u_param * c_zno:.4f}  dr3 0.025 0.025 0.025
po: Zn_BVH  {a_zno/2:.4f}   {a_zno/(2*np.sqrt(3)):.4f}   {1.5*c_zno:.4f}  dr3 0.025 0.025 0.025
po: O_BVH   {a_zno/2:.4f}   {a_zno/(2*np.sqrt(3)):.4f}   {1.5*c_zno - u_param * c_zno:.4f}  dr3 0.025 0.025 0.025
po: Zn_BVH  0.0000   0.0000   {c_zno:.4f}  dr3 0.025 0.025 0.025
po: O_BVH   0.0000   0.0000   {c_zno - u_param * c_zno:.4f}  dr3 0.025 0.025 0.025
#
# Muffin-tin radii (Angstrom)
# Zn-O bond length: ~1.99 A
# Use about 40-45% of bond length for each atom
rm: Zn_BVH  0.85
rm: O_BVH   0.70
#
# z-range for slab
zr: {0.5*c_zno:.2f}  {2.2*c_zno:.2f}
#
sz: 1
sr: 3 0.0 0.0
'''

print("=== ZnO(0001) Bulk File ===")
print(zno_bulk_content)

print("\n=== ZnO(0001) Slab File ===")
print(zno_slab_content)

In [ ]:
# Write the CLEED input files
bulk_file = os.path.join(work_dir, 'ZnO0001.bul')
slab_file = os.path.join(work_dir, 'ZnO0001.inp')

with open(bulk_file, 'w') as f:
    f.write(zno_bulk_content)
    
with open(slab_file, 'w') as f:
    f.write(zno_slab_content)

print(f"Created: {bulk_file}")
print(f"Created: {slab_file}")

In [ ]:
# Convert CLEED files to MTZ format and run Step 1
print("=" * 60)
print("STEP 1: Muffin-Tin Potential Calculation")
print("=" * 60)

try:
    # Load and convert bulk file
    print("\nLoading bulk model from CLEED format...")
    dummycell = model.Unitcell(1, 2, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    bulk_mtz = Converter.import_CLEED(bulk_file, verbose=True)
    print(f"Bulk atoms: {[str(a) for a in bulk_mtz.atoms]}")
    
    print("\nLoading slab model from CLEED format...")
    slab_mtz = Converter.import_CLEED(slab_file, verbose=False)
    print(f"Slab atoms: {[str(a) for a in slab_mtz.atoms]}")
    
    # The MTZ calculation would be done here
    print("\n[Muffin-tin calculation would run here]")
    print("Output: mufftin.d file with potential on radial grid")
    
except Exception as e:
    print(f"\nNote: {e}")
    print("Continuing with demonstration...")

### Understanding the Muffin-Tin Zero (MTZ)

A critical parameter in Step 1 is the **Muffin-Tin Zero (MTZ)** - the average interstitial potential. 

**Important workflow for surfaces:**

1. First run a **bulk calculation** to get the bulk MTZ value
2. Then run a **slab calculation** using the bulk MTZ as input

This is because the vacuum in the slab would otherwise distort the average.

Typical MTZ values:
- Metals: -5 to -15 eV
- Oxides: -8 to -15 eV

## Step 2: Phase Shift Calculation (phsh2)

### Theory

With the muffin-tin potential $V(r)$ from Step 1, we now solve the radial wave equation:

$$\left[ -\frac{\hbar^2}{2m} \frac{d^2}{dr^2} + \frac{\hbar^2 l(l+1)}{2mr^2} + V(r) - E \right] u_l(r) = 0$$

The phase shift $\delta_l(E)$ is extracted from the asymptotic behavior of the solution:

$$u_l(r \to \infty) \propto \sin(kr - l\pi/2 + \delta_l)$$

### Three Calculation Methods

| Program | Method | When to Use |
|---------|--------|-------------|
| `PhSh2cav` | Non-relativistic (Schrodinger) | Light elements (Z < 20) |
| `PhSh2wil` | Non-relativistic with continuity | Light elements, auto-continuous |
| `PhSh2rel` | Relativistic (Dirac) | Heavy elements (Z > 20) |

For ZnO:
- Zinc (Z=30): Relativistic effects are moderate but recommended
- Oxygen (Z=8): Non-relativistic is sufficient

We'll use the relativistic calculation for both for consistency.

In [ ]:
print("=" * 60)
print("STEP 2: Phase Shift Calculation")
print("=" * 60)

# In a real calculation, this would call the Fortran routines:
# from phaseshifts.lib.libphsh import phsh_rel, phsh_cav, phsh_wil

print("\nCalculation type: Relativistic (PhSh2rel)")
print("Reason: Zn (Z=30) has moderate relativistic effects")
print("\nEnergy range: 30 - 400 eV in 5 eV steps")
print("Maximum l: 10")
print("\nOutput files:")
print("  - phasout: Raw phase shifts for all atoms")
print("  - dataph.d: Phase shifts in plottable format")

### Understanding the Raw Phase Shift Output

The `phasout` file contains phase shifts that may have discontinuities of $\pi$ at certain energies. These arise because:

1. Phase shifts are mathematically defined modulo $\pi$
2. At resonances, the phase can jump rapidly

These discontinuities must be removed before using the phase shifts in LEED calculations, which is done in Step 3.

## Step 3: Removing Pi-Jumps (phsh3 / conphas)

### Theory

LEED programs interpolate phase shifts between energy points. Discontinuities would cause:
- Incorrect interpolation
- Wrong diffraction intensities
- Failed structure optimization

The `conphas` (continuous phase) routine:
1. Detects jumps of approximately $\pm \pi$
2. Adds/subtracts $\pi$ to make the function continuous
3. Reformats output for LEED programs

In [ ]:
print("=" * 60)
print("STEP 3: Removing Pi-Jumps (conphas)")
print("=" * 60)

# Demonstrate the Conphas class
print("\nThe Conphas class provides:")
print("  - split_phasout(): Split multi-atom phasout into individual files")
print("  - calculate(): Remove pi-jumps and create continuous phases")
print("  - set_format(): Choose output format (cleed, curve, viperleed)")

# Show example usage
print("\nExample usage:")
print(">>> from phaseshifts.conphas import Conphas")
print(">>> conphas = Conphas(")
print("...     input_files=['Zn.ph', 'O.ph'],")
print("...     output_file='ZnO_phases.phs',")
print("...     formatting='cleed',")
print("...     lmax=10")
print("... )")
print(">>> conphas.calculate()")

## Visualizing ZnO Phase Shifts

Let's visualize what ZnO phase shifts typically look like. We'll generate representative curves based on the physics of Zn and O scattering.

In [ ]:
def generate_model_phase_shifts(Z, energy, lmax=10):
    """
    Generate simplified model phase shifts for demonstration.
    
    This is NOT a physical calculation - just for illustration!
    Real phase shifts require solving the Schrodinger/Dirac equation.
    
    Parameters:
    -----------
    Z : int
        Atomic number
    energy : array
        Energy values in eV
    lmax : int
        Maximum angular momentum
    
    Returns:
    --------
    phases : array
        Phase shifts shape (nE, lmax+1)
    """
    k = np.sqrt(energy / 13.6)  # Wavevector in atomic units
    phases = np.zeros((len(energy), lmax + 1))
    
    for l in range(lmax + 1):
        # Model: phase shift decreases with l, increases with Z
        # Add energy dependence and some resonance-like features
        delta = (Z / 30) * np.arctan(2.0 / (k + l/2 + 1))
        
        # Add d-wave resonance for transition metals (l=2)
        if l == 2 and Z > 20:
            # Shape resonance around 100-200 eV
            resonance = 0.5 * np.exp(-((energy - 120) / 50)**2)
            delta += resonance
        
        # Add some l-dependent baseline
        delta += l * 0.3 * np.exp(-k / 3)
        
        phases[:, l] = delta
    
    return phases

# Generate model phase shifts for Zn and O
energy = np.linspace(30, 400, 75)  # Energy range matching our calculation

phases_Zn = generate_model_phase_shifts(30, energy)  # Zinc, Z=30
phases_O = generate_model_phase_shifts(8, energy)    # Oxygen, Z=8

# Angular momentum labels
l_labels = ['s', 'p', 'd', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n']

In [ ]:
# Create comprehensive phase shift visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = plt.cm.viridis(np.linspace(0, 0.9, 8))

# Plot Zn phase shifts
ax = axes[0, 0]
for l in range(8):
    ax.plot(energy, phases_Zn[:, l], color=colors[l], 
            label=f'l={l} ({l_labels[l]})', linewidth=2)
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Phase Shift (radians)')
ax.set_title('Zinc (Z=30) Phase Shifts', fontsize=14)
ax.legend(loc='upper right', ncol=2, fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Plot O phase shifts
ax = axes[0, 1]
for l in range(6):  # O has fewer significant l values
    ax.plot(energy, phases_O[:, l], color=colors[l], 
            label=f'l={l} ({l_labels[l]})', linewidth=2)
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Phase Shift (radians)')
ax.set_title('Oxygen (Z=8) Phase Shifts', fontsize=14)
ax.legend(loc='upper right', ncol=2, fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Compare s-wave (l=0) for both elements
ax = axes[1, 0]
ax.plot(energy, phases_Zn[:, 0], 'b-', linewidth=2, label='Zn (Z=30)')
ax.plot(energy, phases_O[:, 0], 'r-', linewidth=2, label='O (Z=8)')
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Phase Shift (radians)')
ax.set_title('s-wave (l=0) Comparison', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Compare d-wave (l=2) for both elements
ax = axes[1, 1]
ax.plot(energy, phases_Zn[:, 2], 'b-', linewidth=2, label='Zn (Z=30)')
ax.plot(energy, phases_O[:, 2], 'r-', linewidth=2, label='O (Z=8)')
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Phase Shift (radians)')
ax.set_title('d-wave (l=2) Comparison - Note Zn resonance', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(work_dir, 'zno_phase_shifts.png'), dpi=150)
plt.show()

print(f"\nPhase shift plots saved to: {os.path.join(work_dir, 'zno_phase_shifts.png')}")

## Understanding the Results

### Key Observations

1. **Heavier atoms (Zn) have larger phase shifts** than lighter atoms (O)
   - More electrons = stronger scattering potential

2. **Phase shifts generally decrease with increasing l**
   - Higher l partial waves have centrifugal barriers that reduce scattering

3. **Phase shifts decrease with increasing energy**
   - Higher energy electrons are less affected by the atomic potential

4. **d-wave (l=2) resonances** are common in transition metals like Zn
   - Related to unfilled or just-filled d-orbitals

### Physical Interpretation

The phase shift $\delta_l(E)$ tells us:
- How much the partial wave with angular momentum $l$ is shifted in phase
- Positive phase shift: wavefunction is pulled into the atom (attractive potential)
- The larger $|\delta_l|$, the stronger the scattering in that channel

## Output Formats for LEED Programs

The `phaseshifts` package supports several output formats:

### 1. CLEED Format (.phs)

Used by the CLEED package:
```
nE  lmax
E1  delta_0  delta_1  ...  delta_lmax
E2  delta_0  delta_1  ...  delta_lmax
...
```

### 2. Curve Format (.cur)

Simple format for plotting:
```
E1  delta_l
E2  delta_l
...
```

### 3. ViPErLEED Format

Used by the Vienna LEED package with additional metadata.

In [ ]:
# Demonstrate output format generation
print("=== CLEED Format Example ===")
print(f"{len(energy)}  {8}  # nE  lmax")
for i in [0, 1, -1]:  # First two and last energy
    line = f"{energy[i]:8.2f}"
    for l in range(8):
        line += f"  {phases_Zn[i, l]:7.4f}"
    print(line)
    if i == 1:
        print("...")

print("\n=== Curve Format Example (l=0 only) ===")
for i in [0, 1, 2, -2, -1]:
    if i == 2:
        print("...")
        continue
    print(f"{energy[i]:8.2f}  {phases_Zn[i, 0]:8.5f}")

## Integration with cleedpy

The `phaseshifts` package is designed to work with [cleedpy](https://github.com/empa-scientific-it/cleedpy), enabling:

1. **YAML input format**: Define structures in human-readable YAML
2. **Automatic phase shift generation**: Integrate into LEED fitting workflows
3. **Search optimization**: Embed in structure refinement loops

### Example cleedpy-style YAML input

```yaml
name: ZnO(0001)-Zn
lattice:
  a: 3.25
  c: 5.21
  structure: wurtzite
surface:
  miller_indices: [0, 0, 0, 1]
  termination: Zn
atoms:
  - element: Zn
    position: [0.0, 0.0, 0.0]
    muffin_tin_radius: 0.85
  - element: O
    position: [0.0, 0.0, 1.99]
    muffin_tin_radius: 0.70
energy_range:
  start: 30
  stop: 400
  step: 5
maximum_angular_momentum: 10
```

## Summary

In this tutorial, we covered:

### The Four Steps of phshift2007

| Step | Program | Input | Output | Physics |
|------|---------|-------|--------|---------|
| 0 | phsh0 | atorb | at_*.i | Atomic charge densities |
| 1 | phsh1 | cluster.i, atomic.i | mufftin.d | Muffin-tin potential |
| 2 | phsh2* | mufftin.d | phasout | Raw phase shifts |
| 3 | conphas | phasout | *.phs | Continuous phase shifts |

### Key Parameters for ZnO

| Parameter | Zn | O |
|-----------|-----|---|
| Atomic number | 30 | 8 |
| Muffin-tin radius | 0.85 A | 0.70 A |
| Calculation method | Relativistic | Non-relativistic OK |

### Best Practices

1. Always run bulk calculation first to get the correct MTZ
2. Use relativistic calculations for Z > 20
3. Check that muffin-tin spheres don't overlap
4. Verify phase shifts are continuous before using in LEED

### Next Steps

- Apply these phase shifts in a LEED-IV analysis
- Compare calculated and experimental diffraction curves
- Use structure optimization to refine atomic positions

In [ ]:
# Print summary of files created
print("=" * 60)
print("FILES CREATED IN THIS TUTORIAL")
print("=" * 60)
print(f"\nWorking directory: {work_dir}")
print("\nInput files:")
print(f"  - {os.path.basename(bulk_file)} (bulk geometry)")
print(f"  - {os.path.basename(slab_file)} (slab geometry)")
print(f"  - atorb_Zn.txt (Zn atomic configuration)")
print(f"  - atorb_O.txt (O atomic configuration)")
print("\nOutput plots:")
print("  - zno_structure.png (crystal structure visualization)")
print("  - zno_phase_shifts.png (phase shift curves)")
print("\nDelete working directory when done:")
print(f"  rm -rf {work_dir}")